<a href="https://colab.research.google.com/github/rojinadeuja/NLP-Model-Implementations/blob/master/Word2vec_using_Keras_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
